In [ ]:
import os
import sys
from datetime import date
import warnings
warnings.filterwarnings("ignore")

import numpy as np

sys.path.append('../..') # for running tests py-script
sys.path.append('../../..') # for running this notebook directly
from seismiqb import SeismicGeometry
from seismiqb.src.geometry import export

In [ ]:
""" The following constants parametrize the behavior of the notebook:

DATESTAMP : str
    Execution date in "YYYY-MM-DD" format.
    Used for saving data.
OUTPUT_DIR : str
    Path to the directory for saving data.

You can manage parameters which relates to a cube generation:

CUBE_SHAPE : sequence of three integers
    Shape of a synthetic cube.
SEED: int or None
    Seed used for creation of random generator (check out `np.random.default_rng`).
"""
# Workspace constants
DATESTAMP = date.today().strftime("%Y-%m-%d")
OUTPUT_DIR = './geometry_test_files/'

# Data creation parameters
CUBE_NAME = f'test_cube_{DATESTAMP}.sgy'
CUBE_SHAPE = (1000, 200, 400)
SEED = 42

# Data formats for which running tests
FORMATS = ['hdf5', 'qhdf5', 'blosc', 'qblosc']

In [ ]:
CUBE_PATH = os.path.join(OUTPUT_DIR, 'tmp/', CUBE_NAME)

# Preparation

## Create a fake cube

In [ ]:
rng = np.random.default_rng(SEED)
data_array = rng.normal(0, 1000, CUBE_SHAPE).astype(np.float32)

In [ ]:
with open(os.path.join(OUTPUT_DIR, f'tmp/test_array_{DATESTAMP}.npy'), 'wb') as outfile:
    np.save(outfile, data_array)

In [ ]:
%%time
export.make_segy_from_array(
    array=data_array, path_segy=CUBE_PATH, zip_segy=False,
    sample_rate=2., delay=50, pbar='t'
)

In [ ]:
%%time
geometry_sgy = SeismicGeometry(
    path=CUBE_PATH, process=True, collect_stats=True, spatial=True, pbar='t'
)

## Check data loading

In [ ]:
%%time
_ = SeismicGeometry(
    path=CUBE_PATH,
    headers=SeismicGeometry.HEADERS_POST_FULL,
    index_headers=SeismicGeometry.INDEX_CDP
)

In [ ]:
%%time
_ = SeismicGeometry(
    path=CUBE_PATH,
    headers=SeismicGeometry.HEADERS_POST_FULL,
    index_headers=SeismicGeometry.INDEX_POST
)

## Conversion

In [ ]:
for f in FORMATS:
    _ = geometry_sgy.convert(format=f, quantize=False, store_meta=False, pbar='t')